In [2]:
import pandas as pd

pollution = pd.read_csv('./data/pollution_us_2000_2016.csv.tar.gz')
pollution = pollution.drop(pollution.columns[0], axis=1)

# Is the Dataset tidy?

In [3]:
pollution.tail(10)

,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,NO2 Mean,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
1746651,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-29,Parts per billion,2.564706,...,Parts per billion,0.120000,0.4,8,NaN,Parts per million,0.045625,0.071,4,NaN
1746652,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-29,Parts per billion,2.564706,...,Parts per billion,0.120000,0.4,8,NaN,Parts per million,0.006667,0.100,0,1.0
1746653,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-30,Parts per billion,1.083333,...,Parts per billion,0.016667,0.1,0,0.0,Parts per million,0.101042,0.134,18,NaN
1746654,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-30,Parts per billion,1.083333,...,Parts per billion,0.016667,0.1,0,0.0,Parts per million,0.091667,0.100,2,1.0
1746655,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-30,Parts per billion,1.083333,...,Parts per billion,0.000000,0.0,2,NaN,Parts per million,0.101042,0.134,18,NaN
1746656,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-30,Parts per billion,1.083333,...,Parts per billion,0.000000,0.0,2,NaN,Parts per million,0.091667,0.100,2,1.0
1746657,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,0.939130,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.067714,0.127,0,NaN
1746658,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,0.939130,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.100000,0.100,0,1.0
1746659,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,0.939130,...,Parts per billion,0.000000,0.0,5,NaN,Parts per million,0.067714,0.127,0,NaN
1746660,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,0.939130,...,Parts per billion,0.000000,0.0,5,NaN,Parts per million,0.100000,0.100,0,1.0


At first sight, it seems tidy.
Each variable has its seperate column, each measurement per station per day has its own row, and there is only one type of data object (measurement per station) in the dataset.

HOWEVER, there are lots of duplicates! For example, there are 4 observations for Site 3002 for the date 2000-01-01. We double-checked if the dataset was containing additional information (eg. one measurement for morning, noon, evening and night) that did not get correctly parsed, like a datetime object parsed to a date object, but that wasn't the case. We will therefore have to deal with these duplicates first before we do further analysis.

# Invaid Values
There are also a lot of negative values in the "Mean" columns, which have to be dealt with before going on to the duplicate-removal step, because including them in calculations would incorrectly scew the resulting values.

We know that these values are invalid because the concentration of any given pollutant cannot be less than 0 at any given point in time.

To combat those invalid values, we chose the following strategy: All negative values in the "Mean" columns are set to NaN. This is done to exploit the behavior of .mean() in that it ignores NaN values and sets the result to NaN if all inputs are NaN.

Therefore, if we merge the "Mean" columns of duplicate rows in the following step, we hope to have at least one valid "Mean" value for every observation available. Ideally, the amount of NaNs in the de-duplicated dataset gets reduced to an amount where we can remove the remaining NaNs by LOCF or similar techniques. 

In [4]:
import numpy as np

pollutants = ['NO2', 'O3', 'SO2', 'CO']

print('Site Nums with negative values in the mean columns:')
for pollutant in pollutants:
    print('{} Mean'.format(pollutant))
    print(pollution[pollution['{} Mean'.format(pollutant)] < 0].groupby(['Site Num']).groups.keys())

print('Total distinct Site Nums in the whole dataset for reference:')
print(pollution['Site Num'].nunique())

for pollutant in pollutants:
    row_index = pollution['{} Mean'.format(pollutant)] < 0
    pollution.loc[row_index, '{} Mean'.format(pollutant)] = np.nan

    

Site Nums with negative values in the mean columns:
NO2 Mean
dict_keys([3, 5, 9, 10, 14, 37, 67, 306, 1004, 1005, 1007, 1037, 1100, 1127, 3001, 5005, 5632, 9003, 9009])
O3 Mean
dict_keys([])
SO2 Mean
dict_keys([2, 5, 6, 8, 9, 10, 11, 14, 16, 21, 23, 27, 29, 30, 37, 40, 41, 43, 44, 69, 80, 100, 124, 306, 416, 870, 1002, 1003, 1004, 1005, 1010, 1025, 1028, 1035, 1037, 1039, 1100, 1103, 1127, 2002, 2009, 3001, 3006, 4002, 4201, 5005, 5632, 8001, 9009])
CO Mean
dict_keys([2, 4, 8, 9, 10, 21, 37, 41, 100, 306, 416, 1002, 1003, 1004, 1037, 1127, 2002, 2009, 3001, 4006, 5005, 5632])
Total distinct Site Nums in the whole dataset for reference:
110


# Duplicate Handling
As we already saw, there are a lot of duplicates in this dataset.
To deal with them, we shall use the following strategy:
- A duplicate is an observation from the same station on the same date.
- For every pollutant, there are 4 columns (apart from the measurement scale, which seems to be the same for the duplicate observations):
    - "&lt;pollutant&gt; Mean" is, well, the mean of this day's measurements. To unify those, we will take the mean of any non-NaN measurements of the duplicate rows (the mean of means so to say).
    - "&lt;pollutant&gt; 1st Max Value" is the maximum measured value of that day. To merge this column, we will take the maximum of the duplicate values (to pick the "actual" highest value of that day).
    - "&lt;pollutant&gt; 1st Max Hour" is the hour in which the maximum value was measured. Here we take the value of the observation from which we sourced the maximum "&lt;pollutant&gt; 1st Max Value".
    - "&lt;pollutant&gt; AQI" is the so-called Air Quality Index, which is a non-linear scale and should probably not be thoughtlessly averaged. We will therefore calculate it anew for the merged duplicate observations.

In [5]:
# We needed some performance-efficient way of taking the max of "1st Max Value" while also 
# keeping the accompanying "1st Max Hour", which turned out to be quite difficult

# The most efficient solution was coupling the two values into a tuple column,
# max-aggregating this column and then splitting it back up into two separate columns
pollutants = ['NO2', 'O3', 'SO2', 'CO']

for pollutant in pollutants:
    pollution[pollutant + ' Zipped'] = list(zip(pollution[pollutant + ' 1st Max Value'], pollution[pollutant + ' 1st Max Hour']))

# Group by all the non-pollutant columns and aggregate all but the AQI
pollution_grouped = pollution.groupby(['Site Num', 'Date Local', 'State', 'County', 'City', 'Address'])
pollution_dedup = pollution_grouped.agg({
    'NO2 Mean': 'mean',
    'NO2 Zipped': 'max',
    'O3 Mean': 'mean',
    'O3 Zipped': 'max',
    'SO2 Mean': 'mean',
    'SO2 Zipped': 'max',
    'CO Mean': 'mean',
    'CO Zipped': 'max',
}).reset_index()

# Split the tuple-columns back into two separate 
for pollutant in ['NO2', 'O3', 'SO2', 'CO']:
    pollution_dedup[[pollutant + ' 1st Max Value', pollutant + ' 1st Max Hour']] = pollution_dedup[pollutant + ' Zipped'].apply(pd.Series)

In [6]:
# Drop the tibble-columns, they are no longer needed
pollution_dedup.drop(['NO2 Zipped', 'O3 Zipped', 'SO2 Zipped', 'CO Zipped'], axis=1, inplace=True)
pollution_dedup

,Site Num,Date Local,State,County,City,Address,NO2 Mean,O3 Mean,SO2 Mean,CO Mean,NO2 1st Max Value,NO2 1st Max Hour,O3 1st Max Value,O3 1st Max Hour,SO2 1st Max Value,SO2 1st Max Hour,CO 1st Max Value,CO 1st Max Hour
0,1,2000-01-01,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",10.913043,0.031625,2.064907,0.603382,20.0,0.0,0.043,9.0,3.0,4.0,1.100,8.0
1,1,2000-01-02,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",10.869565,0.026833,1.748136,0.560145,35.0,18.0,0.040,9.0,2.0,2.0,1.400,18.0
2,1,2000-01-03,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",27.782609,0.011333,2.172671,1.174003,45.0,17.0,0.022,9.0,3.0,20.0,2.800,20.0
3,1,2000-01-04,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",33.869565,0.009417,2.992546,1.249003,58.0,17.0,0.021,8.0,5.0,16.0,2.600,20.0
4,1,2000-01-05,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",34.181818,0.011875,3.132575,1.588258,50.0,18.0,0.027,10.0,4.0,7.0,4.000,7.0
5,1,2000-01-06,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",31.318182,0.011292,2.844697,1.389584,51.0,20.0,0.024,8.0,4.0,8.0,3.400,8.0
6,1,2000-01-07,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",30.608696,0.011500,3.222671,1.176359,43.0,8.0,0.025,9.0,12.0,11.0,2.400,7.0
7,1,2000-01-08,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",31.695652,0.008083,3.985715,1.488497,48.0,11.0,0.018,9.0,6.0,10.0,3.300,19.0
8,1,2000-01-09,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",33.043478,0.013417,2.215838,1.314221,57.0,18.0,0.030,10.0,5.0,12.0,2.600,20.0
9,1,2000-01-10,California,San Diego,Chula Vista,"80 E. 'J' ST., CHULA VISTA",35.304348,0.015292,2.503726,1.372192,50.0,19.0,0.037,10.0,4.0,10.0,2.400,19.0


# Missing Value Analysis
After the merging of duplicates is done, we will have to check for and deal with any remaining missing values from the "Mean" columns (as we initially set all negative=invalid values to NaN).

A short analysis shows that we did not get the result we hoped for. The total amount of missing values has of course declined, but the relative amount of missing values has not. This implies that for most original observations with negative "Mean" values, all duplicates had those negative values.

In [7]:
# Get all rows that contain at least one NaN
missings_orig = pollution[pollution[['NO2 Mean', 'O3 Mean', 'SO2 Mean', 'CO Mean']].isna().any(axis=1)]

# How many observations with missings are there?
print("pollution: Number of rows with missings: " + str(len(missings_orig)))
print("pollution: Number of rows in the dataset: " + str(len(pollution)))

# After merging the duplicate rows, there seem to be no missings left
print(missings_orig.count(axis = 0).rsub(len(missings_orig)))

# Same for the dedup-dataset
missings = pollution_dedup[pollution_dedup.isna().any(axis=1)]
print("\npollution_dedup: Number of rows with missings: " + str(len(missings)))
print("pollution_dedup: Number of rows in the dataset: " + str(len(pollution_dedup)))
print(missings.count(axis = 0).rsub(len(missings)))



pollution: Number of rows with missings: 24464
pollution: Number of rows in the dataset: 1746661
State Code               0
County Code              0
Site Num                 0
Address                  0
State                    0
County                   0
City                     0
Date Local               0
NO2 Units                0
NO2 Mean               896
NO2 1st Max Value        0
NO2 1st Max Hour         0
NO2 AQI                  0
O3 Units                 0
O3 Mean                  0
O3 1st Max Value         0
O3 1st Max Hour          0
O3 AQI                   0
SO2 Units                0
SO2 Mean             22690
SO2 1st Max Value        0
SO2 1st Max Hour         0
SO2 AQI              11541
CO Units                 0
CO Mean               1064
CO 1st Max Value         0
CO 1st Max Hour          0
CO AQI               12256
NO2 Zipped               0
O3 Zipped                0
SO2 Zipped               0
CO Zipped                0
dtype: int64

pollution_dedup: Number o

In [8]:
missings

,Site Num,Date Local,State,County,City,Address,NO2 Mean,O3 Mean,SO2 Mean,CO Mean,NO2 1st Max Value,NO2 1st Max Hour,O3 1st Max Value,O3 1st Max Hour,SO2 1st Max Value,SO2 1st Max Hour,CO 1st Max Value,CO 1st Max Hour
16495,2,2012-07-23,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,4.926087,0.022917,NaN,0.265942,7.2,9.0,0.034,10.0,0.1,14.0,0.300,8.0
16503,2,2012-07-25,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,4.643478,0.026458,NaN,0.278805,7.0,6.0,0.038,11.0,0.2,14.0,0.300,5.0
16507,2,2012-07-26,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,4.952174,0.024458,NaN,0.244475,8.2,9.0,0.031,11.0,0.1,10.0,0.300,6.0
16511,2,2012-07-27,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,4.408696,0.021792,NaN,0.251177,8.4,21.0,0.029,9.0,0.3,13.0,0.300,13.0
16515,2,2012-07-28,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,3.482609,0.020375,NaN,0.248731,4.8,0.0,0.031,9.0,0.2,5.0,0.300,0.0
16519,2,2012-07-29,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,3.382609,0.021542,NaN,0.240489,7.2,19.0,0.034,10.0,0.1,13.0,0.300,15.0
16539,2,2012-08-03,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,6.565217,0.021583,NaN,0.272282,8.4,5.0,0.033,10.0,0.2,0.0,0.300,6.0
16543,2,2012-08-04,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,3.752174,0.020458,NaN,0.263858,6.6,9.0,0.026,10.0,-0.1,2.0,0.300,8.0
16850,2,2012-10-22,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,3.791304,0.030833,NaN,0.268025,9.5,7.0,0.037,10.0,0.1,5.0,0.400,8.0
16886,2,2012-10-31,California,Contra Costa,Concord,2956-A TREAT BOULEVARD,4.852174,0.012375,NaN,0.242482,9.6,23.0,0.017,8.0,0.1,8.0,0.300,6.0


# Missing Value Handling

In [21]:
# Excluding 3 measuring stations that have almost a year of misssings, 
# we can LOCF with a limit of 27 days and fill all missings
pollution_dedup_grp = pollution_dedup[~pollution_dedup['Site Num'].isin([1005, 1035, 5632])].groupby('Site Num', as_index=False)
pollution_dedup2 = pollution_dedup_grp.fillna(method='ffill', limit=27)

missings2 = pollution_dedup2[pollution_dedup2.isna().any(axis=1)]
print("\npollution_dedup: Number of rows with missings: " + str(len(missings2)))


pollution_dedup: Number of rows with missings: 0


In [10]:
# --- Recalculate the AQI ---
# This currently fails because there are still negative values in the mean-entries

# import aqi
# pollution_dedup.iloc[102281]
# pollution_dedup.head(200000).apply(lambda x: aqi.to_iaqi(aqi.POLLUTANT_SO2_1H, x['SO2 Mean']), axis=1)